### Importing librarys

In [53]:
import pandas as pd
import os
import matplotlib.pyplot as plt



### Concatenate all 12 files of data in one

In [2]:
files = [file for file in os.listdir("./Sales_Data/")]

annual_data = pd.DataFrame()

for file in files:
    annual_data = pd.concat([annual_data, pd.read_csv("./Sales_Data/"+file)])

annual_data.to_csv("all_months_data.csv", index=False)

### Read the new file with all 12 months of data
- Why not use annual_data from above?

In [3]:
all_data = pd.read_csv("all_months_data.csv")
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,04/19/19 08:46,"917 1st St, Dallas, TX 75001"
1,NaN,NaN,NaN,NaN,NaN,NaN
2,176559,Bose SoundSport Headphones,1,99.99,04/07/19 22:30,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,04/12/19 14:38,"669 Spruce St, Los Angeles, CA 90001"


### Finding out what was the best month of sales and how much was earned in that month
- Why use the annual csv file instead of the files of each month?

##### Cleaning the data, excluding rows with null values and invalid values

In [35]:
all_data = all_data.drop(all_data[all_data["Order ID"] == "Order ID"].index)

all_data.dropna(inplace=True)

all_data = all_data.reset_index(drop=True)

all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Total Price
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",April,23.90
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",April,99.99
2,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",April,600.00
3,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",April,11.99
4,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",April,11.99


#### Parsing the column "Order date" to date time format

In [9]:
all_data["Order Date"] = pd.to_datetime(all_data["Order Date"])

C:\Users\mikael.gualdi\AppData\Local\Temp\ipykernel_18480\3842191188.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])


#### Creating a new column "month"

In [36]:
all_data["Month"] = all_data["Order Date"].dt.month_name()

all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address,Month,Total Price
0,176558,USB-C Charging Cable,2,11.95,2019-04-19 08:46:00,"917 1st St, Dallas, TX 75001",April,23.90
1,176559,Bose SoundSport Headphones,1,99.99,2019-04-07 22:30:00,"682 Chestnut St, Boston, MA 02215",April,99.99
2,176560,Google Phone,1,600.00,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",April,600.00
3,176560,Wired Headphones,1,11.99,2019-04-12 14:38:00,"669 Spruce St, Los Angeles, CA 90001",April,11.99
4,176561,Wired Headphones,1,11.99,2019-04-30 09:27:00,"333 8th St, Los Angeles, CA 90001",April,11.99


#### Converting columns "Price Each" and "Quantity Ordered" to numeric

In [49]:
all_data["Price Each"] = pd.to_numeric(all_data["Price Each"])
all_data["Quantity Ordered"] = pd.to_numeric(all_data["Quantity Ordered"])

#### Creating a new column "total price" (price each * quantity ordered)

In [24]:
all_data["Total Price"] = all_data["Quantity Ordered"] * all_data["Price Each"]

#### New data frame with the data I'm looking for

In [69]:
sales_by_month = all_data.groupby("Month").agg({"Total Price": "sum"})

sales_by_month.sort_values(by="Total Price", ascending=False, inplace=True)

sales_by_month.reset_index()



,Month,Total Price
0,December,4613443.34
1,October,3736726.88
2,April,3390670.24
3,November,3199603.20
4,May,3152606.75
5,March,2807100.38
6,July,2647775.76
7,June,2577802.26
8,August,2244467.88
9,February,2202022.42


#### Ploting sales by month

In [70]:
plt.bar(sales_by_month["Month"], sales_by_month["Total Price"])

KeyError: 'Month'

### Finding out what city had the highest number of sales

### Finding out what products are most often sold together

### Finding out what product sold the most